# XML DOM

Python标准库xml模块提供4个功能模块：

    
    sax    -- 基于事件的XML文档处理；
    dom   -- DOM接口实现；
    etree  -- 文档树
    parsers --  XML解析器的Python包装器


## sax功能模块

In [1]:
import xml.sax
# help(xml.sax)

### sax API结构

sax提供四个模块来处理xml：
       
        xml.sax.xmlreader    — xml的解析接口规范，抽象的，需要继承实现才能使用。
        xml.sax.handler       — 回调处理器，如果与xmlreader绑定，当满足xml条件就会被调用，用来处理具体的xml内容。
        xml.sax.saxutils       — 一些数据处理工具，比如实体转换什么的。
        
        xml.parsers.expat     — xmlreader的具体实现，可以直接使用。也可以使用parse函数直接解析，或者使用make_parser返回解析器，或者直接构造ExpatParser解析器使用。【见下面例子】
        
下面对核心模块xmlreader与handler进一步说明。

####  xmlreader API结构
        
        提供如下几个类实现数据分析：
                |- AttributesImpl
                        |- AttributesNSImpl
                |- InputSource
                |- Locator
                |- XMLReader
                        |- IncrementalParser

#### handler API结构

        提供4个Handler来处理xml内容。
                |-  ContentHandler
                |-  DTDHandler
                |-  EntityResolver
                |-  ErrorHandler

### sax解析器的三种使用方式

#### sax编程模式

In [2]:
# coding = utf-8
import xml.sax.handler


class MyHandler(xml.sax.handler.ContentHandler, xml.sax.handler.DTDHandler):
    def setDocumentLocator(self, locator):
        print('setDocumentLocator')

    def startDocument(self):
        print('startDocument')

    def endDocument(self):
        print('endDocument')

    def startPrefixMapping(self, prefix, uri):
        print('startPrefixMapping')

    def endPrefixMapping(self, prefix):
        print('endPrefixMapping')

    def startElement(self, name, attrs):
        print('startElement')

    def endElement(self, name):
        print('endElement')

    def startElementNS(self, name, qname, attrs):
        print('startElementNS')

    def endElementNS(self, name, qname):
        print('endElementNS')

    def characters(self, content):
        print('characters')

    def ignorableWhitespace(self, whitespace):
        print('ignorableWhitespace')

    def processingInstruction(self, target, data):
        print('processingInstruction')

    def skippedEntity(self, name):
        print('skippedEntity')

    def notationDecl(self, name, publicId, systemId):
        print('notationDecl')

    def unparsedEntityDecl(self, name, publicId, systemId, ndata):
        print('unparsedEntityDecl')


class MyError(xml.sax.ErrorHandler):

    def error(self, exception):
        print('error')
        return super().error(exception)

    def fatalError(self, exception):
        print('fatalError')
        return super().fatalError(exception)

    def warning(self, exception):
        print('warning')
        super().warning(exception)


xml.sax.parse(
    source='codes/note.xml',
    handler=MyHandler(),
    errorHandler=MyError())


setDocumentLocator
startDocument
startElement
characters
characters
startElement
characters
endElement
characters
characters
startElement
characters
endElement
characters
characters
startElement
characters
endElement
characters
characters
startElement
characters
endElement
characters
endElement
endDocument


#### 构造parser       

    注意：不能直接构造XMLReader，而是使用工厂模式函数：xml.sax.make_parser()来产生parser，要么直接调用parse函数解析处理xml。
    
    XMLReader是xml解析的接口规范，其实现就是：xml.sax.expatreader.ExpatParser

In [3]:
# coding = utf-8
import xml.sax.handler
import xml.sax.xmlreader

class MyHandler(xml.sax.handler.ContentHandler,xml.sax.handler.DTDHandler):
    def setDocumentLocator(self, locator):
        print('setDocumentLocator')

class MyError(xml.sax.ErrorHandler):

    def error(self, exception):
        print('error')
        return super().error(exception)

'''
xml.sax.parse(
    source='books.xml',
    handler=MyHandler(),
    errorHandler=MyError())
'''
handler = MyHandler()
error = MyError()
parser = xml.sax.make_parser()
print(parser)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/note.xml')


setDocumentLocator


#### ExpatParser的使用
    
    ExpatParser实现的接口类：
    class ExpatParser(xml.sax.xmlreader.IncrementalParser, xml.sax.xmlreader.Locator)
    直接

In [4]:
# coding = utf-8
import xml.sax.handler
import xml.sax.xmlreader
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler,xml.sax.handler.DTDHandler):
    def setDocumentLocator(self, locator):
        print('setDocumentLocator')



class MyError(xml.sax.ErrorHandler):

    def error(self, exception):
        print('error')
        return super().error(exception)

'''
xml.sax.parse(
    source='books.xml',
    handler=MyHandler(),
    errorHandler=MyError())
'''
handler = MyHandler()
error = MyError()
# parser = xml.sax.make_parser()
parser = xml.sax.expatreader.ExpatParser(namespaceHandling=True)
print(parser)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/note.xml')


setDocumentLocator


### handler的使用

    对DTDHandler这里不说明，

#### Locator的使用
    
        Locator是文档定位，其实现类是ExpatLocator类。
            |-  xml.sax.xmlreader.Locator(builtins.object)
                    |-  xml.sax.expatreader.ExpatLocator
        
        该类使用是个函数提供四个信息：
             |  - getColumnNumber(self)     xml当前列数
             |  - getLineNumber(self)          xml当前行数
             |  - getPublicId(self)                xml的public ID：在Python中通常是None
             |  - getSystemId(self)              xml的system ID：通常就是xml文件




In [5]:
# coding = utf-8
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler):
    def setDocumentLocator(self, locator):
        print(f'system ID：{locator.getSystemId()}')
        print(f'public ID：{locator.getPublicId()}')
        print('行列位置(%d,%d)' % (locator.getLineNumber(),locator.getColumnNumber()))


handler = MyHandler()
error = xml.sax.ErrorHandler()

parser = xml.sax.expatreader.ExpatParser(namespaceHandling=True)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/web.xml')


system ID：codes/web.xml
public ID：None
行列位置(1,0)


#### startDocument与endDocument的文档解析事件函数
        
        在文档解析开始的时候触发解析事件函数：startDocument，在文档解析结束的时候触发解析事件函数：endDocument。


In [6]:
# coding = utf-8
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler):

    def startDocument(self):
        print('开始文档解析')

    def endDocument(self):
        print('文档解析结束')


handler = MyHandler()
error = xml.sax.ErrorHandler()

parser = xml.sax.expatreader.ExpatParser(namespaceHandling=True)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/books.xml')


开始文档解析
文档解析结束


#### startElement与endElement元素解析事件函数        
    
        startElement函数在每个元素开始前触发。     
        endElement函数在每个元素结束后触发。     
        函数的原型如下：   
            |- def startElement(self, name, attrs):      
                    |-  name：类型是字符串str，表示元素名。    
                    |-  attrs：类型是xml.sax.xmlreader.AttributesImpl，表示元素的所有属性。    
            
            |- def endElement(self, name):    
            
       
         其中attrs是属性列表，本质是一个字典，还有一些其他成员，可以使用帮助方式查看。
         

In [7]:
# coding = utf-8
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler):

    def startElement(self, name, attrs):
        print('元素解析开始，元素名：', name)
        # attrs是属性列表
        print(type(attrs), type(name))
        # 属性列表
        print('属性数：%d' % attrs.getLength())
        for item in attrs.items():
            print('属性名：%s，属性值：%s' % item)


    def endElement(self, name):
        print('元素解析结束，元素名：', name)


handler = MyHandler()
error = xml.sax.ErrorHandler()

# namespaceHandling 对具有命名空间的元素处理，没有命名空间设置围为0。
parser = xml.sax.expatreader.ExpatParser(namespaceHandling=0)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/books.xml')


元素解析开始，元素名： books
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：0
元素解析开始，元素名： book
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：1
属性名：category，属性值：Python
元素解析开始，元素名： title
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：1
属性名：lang，属性值：zh
元素解析结束，元素名： title
元素解析开始，元素名： author
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：0
元素解析结束，元素名： author
元素解析开始，元素名： year
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：0
元素解析结束，元素名： year
元素解析开始，元素名： price
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：0
元素解析结束，元素名： price
元素解析开始，元素名： publisher
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：0
元素解析结束，元素名： publisher
元素解析结束，元素名： book
元素解析开始，元素名： book
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：1
属性名：category，属性值：系统运维
元素解析开始，元素名： title
<class 'xml.sax.xmlreader.AttributesImpl'> <class 'str'>
属性数：1
属性名：lang，属性值：zh
元素解析结束，元素名： title
元素解析开始，元素名： author
<class 'xml.sax.xmlreader.AttributesImpl'> 

#### characters文本解析事件函数

        函数原型是：characters(self, content)
            参数：content是文本内容。

In [8]:
# coding = utf-8
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler):

    def characters(self, content):
        # 为了看见所有的字符，下面输出采用子节序列。
        # 下面的文本包含所有元素结束的换行字符。
        print(content.encode('utf-8'))


handler = MyHandler()
error = xml.sax.ErrorHandler()

# namespaceHandling 对具有命名空间的元素处理，没有命名空间设置围为0。
parser = xml.sax.expatreader.ExpatParser(namespaceHandling=0)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/note.xml')


b'\n'
b'    '
b'Tove'
b'\n'
b'    '
b'Jani'
b'\n'
b'    '
b'Reminder'
b'\n'
b'    '
b"Don't forget me this weekend!"
b'\n'


#### startElementNS与endElementNS带命名空间名的元素解析事件函数

        startElementNS(self, name, qname, attrs)
            |- qname：命名空间，qname是qualified name 的简写，比如：<song:id>。
            |- name：与startElement不同，这里是一个元组，格式如下：(uri, localname)
            
            

In [9]:
# coding = utf-8
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler):

    def startElementNS(self, name, qname, attrs):
        print(name, qname)
    def endElementNS(self, name, qname):
        print('结束：', name)
    


handler = MyHandler()
error = xml.sax.ErrorHandler()

# namespaceHandling 对具有命名空间的元素处理，没有命名空间设置围为0。
parser = xml.sax.expatreader.ExpatParser(namespaceHandling=True)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/song.xml')


('http://www.example.org/athena', 'body') None
('http://www.example.org/athena', 'id') None
结束： ('http://www.example.org/athena', 'id')
('http://www.example.org/athena', 'name') None
结束： ('http://www.example.org/athena', 'name')
('http://www.example.org/athena', 'birthday') None
结束： ('http://www.example.org/athena', 'birthday')
结束： ('http://www.example.org/athena', 'body')


#### startPrefixMapping与endPrefixMapping解析事件函数
        
        命名空间前缀映射解析事件。
        startPrefixMapping(self, prefix, uri):
            |- prefix ：前缀
            |- uri：uri
            

In [10]:
# coding = utf-8
import xml.sax.expatreader


class MyHandler(xml.sax.handler.ContentHandler):


    def startPrefixMapping(self, prefix, uri):
        print(prefix, uri)

    def endPrefixMapping(self, prefix):
        print(prefix)


handler = MyHandler()
error = xml.sax.ErrorHandler()

# namespaceHandling 对具有命名空间的元素处理，没有命名空间设置围为0。
parser = xml.sax.expatreader.ExpatParser(namespaceHandling=True)
parser.setContentHandler(handler)
parser.setErrorHandler(error)
parser.setDTDHandler(handler)
parser.parse('codes/song.xml')


song http://www.example.org/athena
xsi http://www.w3.org/2001/XMLSchema-instance
xsi
song


#### 其他
    
        还有一些其他事件函数，这里不详细解释。比如：
        processingInstruction(self, target, data)函数就是处理'<?xml version="1.0" encoding="UTF-8"?>'。但实际不触发该函数。

注意：
    SAX解析处理XML比较麻烦，我们都不直接使用，这里只是说明xml的处理底层细节。
    
总结：
    sax模块结构分成两个标准接口定义：handler与xmlreader，一个实现模块expatreader，一个工具模块saxutils。

## dom功能模块

### dom API结构

查看xml.dom模块，可以看到xml.dom的API结构如下。
    
        NodeFilter             # NodeFilter接口，仅仅提供常量定义
        domreg                 # 不直接使用，提供DOM实现访问接口。
        expatbuilder          # 
        minicompat
        minidom                # dom的实现方式之一
        pulldom                 # dom的实现方式之二
        xmlbuilder
    定义了Node结构
        Node
        UserDataHandler



#### domreg  
        
        两个核心函数获取与注册实现工厂。
        
            getDOMImplementation(name=None, features=())    # 得到DOM实现
            registerDOMImplementation(name, factory)            # 注册DOM实现（需要自己实现一个DOMImplementation接口，并注册）
    
        提供两个数据：
            registered = {}
            well_known_implementations = {'4DOM': 'xml.dom.DOMImplementation', 'mi...
        

In [11]:
import xml.dom
import xml.dom.domreg

# 数据
print(xml.dom.domreg.well_known_implementations)
print(xml.dom.domreg.registered)

mini_dom = xml.dom.domreg.getDOMImplementation(name='minidom')
# '4DOM'没有提供实现，下面代码会产生异常。
# four_dom = xml.dom.domreg.getDOMImplementation(name='4DOM')
print(mini_dom)
# print(four_dom)


{'minidom': 'xml.dom.minidom', '4DOM': 'xml.dom.DOMImplementation'}
{}


#### minidom模块

        该模块使用help可以得到帮助，该模块下提供三个函数，负责启动解析工作。
            |- getDOMImplementation(features=None)
            |- parse(file, parser=None, bufsize=None)
                    |-  返回document对象
            |- parseString(string, parser=None)
                    
        
        同时提供对DOM对象的封装：
            |-   builtins.object
                        |-  Childless
                                |-  CharacterData(Childless, Node)
                                |        |-  Comment
                                |        |-  Text
                                |                |-  CDATASection
                                |-  ProcessingInstruction(Childless, Node)
            |-  ElementInfo
            |-  Identified
                    |-  DocumentType(Identified, Childless, Node)
                    |-  Entity(Identified, Node)
                    |-  Notation(Identified, Childless, Node)
            |-  NamedNodeMap
            |-  ReadOnlySequentialNamedNodeMap
            |-  TypeInfo
        
        
            节点对象：
            |-  xml.dom.Node(builtins.object)
                    |-  Node
                            |-  Attr
                            |-  Document(Node, xml.dom.xmlbuilder.DocumentLS)
                            |-  DocumentFragment
                            |-  Element
            
            DOM解析实现：
            xml.dom.xmlbuilder.DOMImplementationLS(builtins.object)
                    |-  DOMImplementation

    具体每个接口的接口函数，可以通过help与dir查看到帮助。


In [12]:
import xml.dom
import xml.dom.minidom
import xml.sax

dom = xml.dom.minidom.parse(
    'note.xml',
    # parser=None,    # parser = xml.sax.make_parser()或者ExpatParser对象
    parser=xml.sax.make_parser(),
    bufsize=1024*10)
print(dom)


#### Node与Element对象提供节点访问与遍历
        |  appendChild(self, node)
        |  
        |  cloneNode(self, deep)
        |  
        |  getInterface(self, feature)
        |  
        |  getUserData(self, key)
        |  
        |  hasChildNodes(self)
        |  
        |  insertBefore(self, newChild, refChild)
        |  
        |  isSameNode(self, other)
        |  
        |  isSupported(self, feature, version)
        |  
        |  normalize(self)
        |  
        |  removeChild(self, oldChild)
        |  
        |  replaceChild(self, newChild, oldChild)
        |  
        |  setUserData(self, key, data, handler)
        |  
        |  toprettyxml(self, indent='\t', newl='\n', encoding=None)
        |  
        |  toxml(self, encoding=None)
        |  
        |  unlink(self)

In [13]:
import xml.dom
import xml.dom.minidom
import xml.sax

dom = xml.dom.minidom.parse(
    'note.xml',
    # parser=None,    # parser = xml.sax.make_parser()或者ExpatParser对象
    parser=xml.sax.make_parser(),
    bufsize=1024*10)
children = dom.childNodes


def list_nodes(nd):
    if nd.nodeType == xml.dom.Node.ELEMENT_NODE:
        print("标签名：", nd.tagName)
        # print('节点名：', nd.nodeName)
        for attr_ in nd.attributes.items():
            print(attr_.name, attr_.value)

    if nd.nodeType == xml.dom.Node.TEXT_NODE:
        if  nd.wholeText.strip() != '':
            print('文本：', nd.wholeText.strip())

    if nd.nodeType == xml.dom.Node.ELEMENT_NODE and nd.hasChildNodes():
        for nd_ in nd.childNodes:
            list_nodes(nd_)


list_nodes(children[0])


标签名： note
标签名： to
文本： Tove
标签名： from
文本： Jani
标签名： heading
文本： Reminder
标签名： body
文本： Don't forget me this weekend!


Document对象提供文档创建操作
        
        |  appendChild(self, node)
        |  
        |  cloneNode(self, deep)
        |  
        |  createAttribute(self, qName)
        |  
        |  createAttributeNS(self, namespaceURI, qualifiedName)
        |  
        |  createCDATASection(self, data)
        |  
        |  createComment(self, data)
        |  
        |  createDocumentFragment(self)
        |  
        |  createElement(self, tagName)
        |  
        |  createElementNS(self, namespaceURI, qualifiedName)
        |  
        |  createProcessingInstruction(self, target, data)
        |        
        |  createTextNode(self, data)
        |  
        |  getElementById(self, id)
        |  
        |  getElementsByTagName(self, name)
        |  
        |  getElementsByTagNameNS(self, namespaceURI, localName)
        |  
        |  importNode(self, node, deep)
        |  
        |  isSupported(self, feature, version)
        |  
        |  removeChild(self, oldChild)
        |  
        |  renameNode(self, n, namespaceURI, name)
        |  
        |  unlink(self)
        |  
        |  writexml(self, writer, indent='', addindent='', newl='', encoding=None)
        

数据定义：

        |  childNodes
        |  
        |  doctype
        |  
        |  documentElement

        

DocumentLS

        |  abort(self)
        |  
        |  load(self, uri)
        |  
        |  loadXML(self, source)
        |  
        |  saveXML(self, snode)

#### DOMImplementation接口实现
    
        implementation = xml.dom.minidom.getDOMImplementation()
            |-  createDocument(self, namespaceURI, qualifiedName, doctype)
            |  
            |-  createDocumentType(self, qualifiedName, publicId, systemId)
            |  
            |-  getInterface(self, feature)
            |  
            |-  hasFeature(self, feature, version)
            
            下面的接口没有实现
            |
            |-  createDOMBuilder(self, mode, schemaType)
            |  
            |-  createDOMInputSource(self)
            |  
            |-  createDOMWriter(self)
            
  由于没有实现createDOMWriter接口，所以python中xml文件的读写使用Document对象提供的writexml函数实现。


In [14]:
# coding = utf-8
import xml.dom
import xml.dom.minidom

implementation = xml.dom.minidom.getDOMImplementation()
print(implementation)
doc_type = implementation.createDocumentType(
    qualifiedName='HH',
    publicId='-//mybatis.org//DTD Config 3.0//EN',
    systemId='http://mybatis.org/dtd/mybatis-3-config.dtd')
doc = implementation.createDocument(
    namespaceURI='http://xmlns.jcp.org/xml/ns/javaee',
    qualifiedName='HH',
    doctype=doc_type)
# writer = implementation.createDOMWriter()   # 没有实现，直接使用文件描述符号。
# 创建根节点
root_element = doc.createElement('myroot')
attr1 = doc.createAttribute('属性')
attr1.value = '属性值'
root_element.setAttributeNode(attr1)
# doc.documentElement只读，创建文档的时候唯一产生
doc.documentElement.appendChild(root_element)

# 产生文本子节点
txt = doc.createTextNode('我是文本')
root_element.appendChild(txt)

with open('my.xml','w') as fd:
    doc.writexml(fd,
                 indent='\t',
                 addindent='\t',
                 newl='\n',
                 encoding='utf-8')


In [15]:
# 下面是读取产生的xml文件
with open('my.xml', 'r') as fd:
    print(fd.read())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE HH
  PUBLIC '-//mybatis.org//DTD Config 3.0//EN'
  'http://mybatis.org/dtd/mybatis-3-config.dtd'>
	<HH>
		<myroot 属性="属性值">我是文本</myroot>
	</HH>



#### 使用Document直接构造xml DOM对象

        Document直接创建的对象，起产生的xml DOM不受documentURI与doctype属性影响。而且不会自动产生documentElement元素。这一点与上面createDocument函数产生的DOM是有差异的。所以建议使用上面的方式产生DOM文档。

In [16]:
# coding = utf-8
import xml.dom
import xml.dom.minidom

implementation = xml.dom.minidom.getDOMImplementation()
print(implementation)
doc_type = implementation.createDocumentType(
    qualifiedName='HH',
    publicId='-//mybatis.org//DTD Config 3.0//EN',
    systemId='http://mybatis.org/dtd/mybatis-3-config.dtd')

# writer = implementation.createDOMWriter()   # 没有实现，直接使用文件描述符号。
doc = xml.dom.minidom.Document()
doc.documentURI='http://xmlns.jcp.org/xml/ns/javaee'
doc.doctype=doc_type
# 创建根节点
root_element = doc.createElement('myroot')
attr1 = doc.createAttribute('属性')
attr1.value = '属性值'
root_element.setAttributeNode(attr1)
# doc.documentElement只读，创建文档的时候唯一产生
doc.appendChild(root_element)

# 产生文本子节点
txt = doc.createTextNode('我是文本')
root_element.appendChild(txt)

with open('your.xml','w') as fd:
    doc.writexml(fd,
                 indent='\t',
                 addindent='\t',
                 newl='\n',
                 encoding='utf-8')


#### expatbuilder

        提供基本的builder实现：
                builtins.object
                        ElementInfo
                        ExpatBuilder
                                FragmentBuilder
                                InternalSubsetExtractor
                        FilterCrutch
                                Rejecter
                                Skipper
                        FilterVisibilityController
                        Namespaces
                                ExpatBuilderNS(Namespaces, ExpatBuilder)
                                FragmentBuilderNS(Namespaces, FragmentBuilder)
                                
        核心的是ExpatBuilder类的两个成员函数：
                parseFile(self, file)
                        返回Document对象
                parseString(self, string)
                        返回Document对象。

In [17]:
import xml.dom
import xml.dom.minidom
import xml.dom.expatbuilder

builder = xml.dom.expatbuilder.ExpatBuilder()
dom = builder.parseFile(open('note.xml', 'r'))
children = dom.childNodes
print(children)
print(dom.documentElement)

def list_nodes(nd):
    if nd.nodeType == xml.dom.Node.ELEMENT_NODE:
        print("标签名：", nd.tagName)
        # print('节点名：', nd.nodeName)
        for attr_ in nd.attributes.items():
            print(attr_.name, attr_.value)

    if nd.nodeType == xml.dom.Node.TEXT_NODE:
        print('文本：', nd.wholeText)

    if nd.nodeType == xml.dom.Node.ELEMENT_NODE and nd.hasChildNodes():
        for nd_ in nd.childNodes:
            list_nodes(nd_)


list_nodes(dom.documentElement)


[<xml.dom.minidom.DocumentType object at 0x103fe1bd0>, <DOM Element: note at 0x103f98178>]
<DOM Element: note at 0x103f98178>
标签名： note
文本： 
    
标签名： to
文本： Tove
文本： 
    
标签名： from
文本： Jani
文本： 
    
标签名： heading
文本： Reminder
文本： 
    
标签名： body
文本： Don't forget me this weekend!
文本： 



#### minicompat

        提供了NodeList接口的定义。
                builtins.list(builtins.object)
                        |- NodeList
                builtins.tuple(builtins.object)
                        |- EmptyNodeList
                        
       这个接口就是所有子节点的返回类型。本质是list类型。

In [18]:
import xml.dom
import xml.dom.minidom
import xml.dom.expatbuilder

builder = xml.dom.expatbuilder.ExpatBuilder()
dom = builder.parseFile(open('note.xml', 'r'))
children = dom.childNodes
print(type(children))

<class 'xml.dom.minicompat.NodeList'>


#### xmlbuilder      
    
        实现了DOM level-3特征。主要如下几个类：
            builtins.object
                    |- DOMBuilder：负责解析过程与处理。
                    |- DOMEntityResolver：负责解析DTD中的实体。
                    |- DOMInputSource：负责xml的数据输入。
        这三个类相互协作解析xml数据。
        
        下面是一个小例子，说明它们之间的协作关系。

In [19]:
import xml.dom
import xml.dom.xmlbuilder

builder = xml.dom.xmlbuilder.DOMBuilder()

input_stream = xml.dom.xmlbuilder.DOMInputSource()
# input_stream.encoding = 'utf-8'
# input_stream.systemId = 'http://mybatis.org/dtd/mybatis-3-config.dtd'
# input_stream.publicId = '-//mybatis.org//DTD Config 3.0//EN'
input_stream.byteStream = open('codes/books.xml', 'r')

result = builder.parse(input_stream)
print(result)
print(result.documentElement.nodeName)


books


####  pulldom     
        
        提供在SAX基础上的DOM处理模型。
                |-  builtins.object
                        |-  DOMEventStream
                        |-  ErrorHandler
                |-  xml.sax.handler.ContentHandler(builtins.object)
                        |-  PullDOM
                                    |-  SAX2DOM
        提供两个函数来封装DOMEventStream的处理细节。
                |-parse(stream_or_string, parser=None, bufsize=None)
                |
                |- parseString(string, parser=None)
        
        两个函数返回DOMEventStream对象，通过getEvent返回事件命名与Document对象。

In [20]:
import xml.dom
import xml.dom.pulldom
import xml.sax.expatreader

# 方式一：
result = xml.dom.pulldom.parse('codes/books.xml')
e, doc = result.getEvent()
print(e)
print(doc.documentElement)

# 方式二：
parser = xml.sax.expatreader.ExpatParser(namespaceHandling=True)
fd = open('codes/books.xml','r')
ds = xml.dom.pulldom.DOMEventStream(fd, parser=parser, bufsize=1024*10)
e_, doc_ = ds.getEvent()
print(e_)


START_DOCUMENT
<DOM Element: books at 0x104001a60>
START_DOCUMENT
